STEP 5: LSTM Model Training and Evaluation
This step loads preprocessed sequences, builds an LSTM model to predict next-hour traffic density, trains and validates the model, and saves the trained model for future inference.

In [ ]:
import numpy as np
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ReduceLROnPlateau


In [ ]:
# Load sequences
X_train = np.load("X_train.npy")
y_train = np.load("y_train.npy")
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")

# Load scaler (optional, for inverse transform later)
with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


In [ ]:
model = Sequential([
    LSTM(96, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.15),

    LSTM(48),
    Dropout(0.15),

    Dense(1)
])

model.compile(
    optimizer="adam",
    loss="mse",
    metrics=["mae"]
)
model.summary()

In [ ]:
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=5,
    min_lr=1e-5
)


In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)


In [ ]:
# Plot Training History
plt.figure(figsize=(10,5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Training Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.legend()
plt.show()


In [ ]:
# Evaluate Model on Test Set
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
print(f"Test MSE: {test_loss:.4f}")
print(f"Test MAE: {test_mae:.4f}")

In [ ]:
# Make Predictions and Inverse Transform
y_pred_scaled = model.predict(X_test)


# Inverse transform (REAL SCALE)
y_pred_inv = scaler.inverse_transform(y_pred_scaled)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))


# =========================
# Plot: Real-scale prediction
# =========================
plt.figure(figsize=(12,6))
plt.plot(y_test_inv, label="Actual Traffic Density")
plt.plot(y_pred_inv, label="Predicted Traffic Density")
plt.title("Traffic Density Prediction (Real Scale)")
plt.xlabel("Time Step")
plt.ylabel("Avg Queue Density")
plt.legend()
plt.show()

In [ ]:
# Predict future traffic density (short-term forecasting)
def predict_future(model, recent_sequence, steps=8):
    future_preds = []
    current_seq = recent_sequence.copy()

    for _ in range(steps):
        pred = model.predict(current_seq.reshape(1, -1, 1))[0][0]
        future_preds.append(pred)
        current_seq = np.append(current_seq[1:], pred)

    return np.array(future_preds)

# Example usage (last test sequence)
future_scaled = predict_future(model, X_test[-1], steps=8)
future_pred = scaler.inverse_transform(future_scaled.reshape(-1, 1))


In [ ]:
import os

os.makedirs("models", exist_ok=True)


In [ ]:
model.save("models/lstm_traffic_model.keras")
print("Trained LSTM model saved successfully.")


Outcome of STEP 5:
An LSTM model was trained to predict next-hour traffic density.
The model was evaluated on a test set and visualized predictions versus actual traffic.
The trained model was saved as lstm_traffic_model.h5 for future inference in the frontend application.